In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('model/fine_tuned_BERT')
tokenizer = AutoTokenizer.from_pretrained("tokenizer")